# Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [168]:
import pandas as pd

In [169]:
data = pd.read_csv('ratings.csv', delimiter=',')
data.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [177]:
#Сколько раз пользователь оценил фильм
time_user = data.pivot_table(index = 'userId', values = 'rating', aggfunc = 'count')
time_user.rename(columns = {'rating': 'count_marks'}, inplace = True)
time_user_fin = time_user[time_user.count_marks > 100]
time_user_fin_ = time_user_fin.groupby('userId').mean().reset_index()
time_user_fin_

,userId,count_marks
0,4,204
1,8,116
2,15,1700
3,17,363
4,19,423
...,...,...
253,656,128
254,659,142
255,664,519
256,665,434


In [171]:
def groupby_time(data):
    return data.timestamp.max() - data.timestamp.min()

In [172]:
#счиатем время жизни пользователя
livetime = data.groupby('userId').apply(groupby_time)
livetime_groupby = livetime.groupby('userId').mean().reset_index()
livetime_groupby

,userId,0
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101
...,...,...
666,667,1014
667,668,282
668,669,685
669,670,2162705


In [176]:
time_user_fin_.merge(livetime_groupby, on = 'userId', how='left')

,userId,count_marks,0
0,4,204,203560
1,8,116,85187
2,15,1700,471393496
3,17,363,8053
4,19,423,5282
...,...,...,...
253,656,128,3053
254,659,142,31609411
255,664,519,98180439
256,665,434,54131471


# Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [16]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [17]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [18]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [19]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [42]:
rzd_grouped = rzd.groupby('client_id').mean()
auto_grouped = auto.groupby('client_id').mean()
air_grouped = air.groupby('client_id').mean()

In [195]:
#Пункт 1 - выводим выручку по каждому клиенту в разбивке на источники
k = rzd_grouped.join(auto_grouped, how='outer')
revenue_table  = k.join(air_grouped, how='outer')
revenue_table_grouped = revenue_table.groupby('client_id').mean().reset_index()
revenue_table_grouped

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [196]:
adress_grouped = client_base.groupby('client_id')
adress_grouped.head()

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [202]:
revenue_table_grouped.join(adress_grouped, on='client_id', how='left')

ValueError: Joining multiple DataFrames only supported for joining on index